In [ ]:
import os
import glob
import cv2
import math
import itertools

import numpy as np
from datetime import datetime
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix

from keras import backend
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, Input
from keras.optimizers import SGD
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

In [ ]:
class DriverDistractionHelper:
    
    def __init__(self, img_rows, img_cols, color_type=1):
        x_train, y_train, driver_id, unique_drivers = self.read_and_normalize_train_data(img_rows, img_cols, color_type)
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, shuffle=True)
        
        x_manual_tests = self.read_and_normalize_manual_test_data(120, 160, color_type)
        
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
        self.x_manual_tests = x_manual_tests
    
    def read_and_normalize_train_data(self, img_rows, img_cols, color_type):
        cache_path = os.path.join('cache', 'train_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
        if not os.path.isfile(cache_path) or use_cache == 0:
            x_train, y_train, driver_id, unique_drivers = self.load_train(img_rows, img_cols, color_type)
            self.cache_data((x_train, y_train, driver_id, unique_drivers), cache_path)
        else:
            print('Restore train from cache!')
            (x_train, train_target, driver_id, unique_drivers) = restore_data(cache_path)

        x_train = np.array(x_train, dtype=np.uint8)
        y_train = np.array(y_train, dtype=np.uint8)
        x_train = x_train.reshape(x_train.shape[0], color_type, img_rows, img_cols)
        y_train = np_utils.to_categorical(y_train, 10)
        x_train = x_train.astype('float32')
        x_train /= 255
        print('Train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        return x_train, y_train, driver_id, unique_drivers
    
    def read_and_normalize_manual_test_data(self, img_rows, img_cols, color_type=1):
        cache_path = os.path.join('cache', 'test_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
        if not os.path.isfile(cache_path) or use_cache == 0:
            test_data = self.load_manual_test(img_rows, img_cols, color_type)
            self.cache_data((test_data), cache_path)
        else:
            print('Restore test from cache!')
            (test_data, test_id) = restore_data(cache_path)

        test_data = np.array(test_data, dtype=np.uint8)
        test_data = test_data.reshape(test_data.shape[0], color_type, img_rows, img_cols)
        test_data = test_data.astype('float32')
        test_data /= 255
        print('Test shape:', test_data.shape)
        print(test_data.shape[0], 'test samples')
        return test_data
    
    def load_train(self, img_rows, img_cols, color_type=1):
        X_train = []
        y_train = []
        driver_id = []

        driver_data = self.get_driver_data()

        print('Read train images')
        for j in range(10):
            print('Load folder c{}'.format(j))
            path = os.path.join('..', 'input', 'train', 'c' + str(j), '*.jpg')
            files = glob.glob(path)
            for fl in files:
                flbase = os.path.basename(fl)
                img = self.get_image(fl, img_rows, img_cols, color_type)
                X_train.append(img)
                y_train.append(j)
                driver_id.append(driver_data[flbase])

        unique_drivers = sorted(list(set(driver_id)))
        print('Unique drivers: {}'.format(len(unique_drivers)))
        print(unique_drivers)
        return X_train, y_train, driver_id, unique_drivers
    
    def load_manual_test(self, img_rows, img_cols, color_type=1):
        print('Read manual test images')
        path = os.path.join('..', 'input', 'test', '*.jpg')
        files = glob.glob(path)
        X_test = []
        total = 0
        for fl in files:
            flbase = os.path.basename(fl)
            img = self.get_image(fl, img_rows, img_cols, color_type)
            X_test.append(img)
            total += 1
        return X_test
    
    def cache_data(self, data, path):
        if os.path.isdir(os.path.dirname(path)):
            file = open(path, 'wb')
            pickle.dump(data, file)
            file.close()
        else:
            print('Directory doesnt exists')
    
    def get_driver_data(self):
        dr = dict()
        path = os.path.join('..', 'input', 'driver_imgs_list.csv')
        print('Read drivers data')
        f = open(path, 'r')
        line = f.readline()
        while (1):
            line = f.readline()
            if line == '':
                break
            arr = line.strip().split(',')
            dr[arr[2]] = arr[0]
        f.close()
        return dr
    
    def get_image(self, path, img_rows, img_cols, color_type=1):
        # Load as grayscale
        if color_type == 1:
            img = cv2.imread(path, 0)
        elif color_type == 3:
            img = cv2.imread(path)
        return cv2.resize(img, (img_cols, img_rows))

In [ ]:
def create_model(img_rows, img_cols, color_type=1):
    return None

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
img_rows = 48
img_cols = 64
color_type = 3

batch_size = 32
nb_epoch = 20
random_state = 51

In [ ]:
ddh = DriverDistractionHelper(img_rows, img_cols, color_type)

In [ ]:
model = create_model(img_rows, img_cols, color_type)

In [ ]:
time_before_training = datetime.now()
model.fit(ddh.x_train, ddh.y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1)
time_after_training = datetime.now()

In [ ]:
time_before_pred = datetime.now()
predictions_valid = model.predict(ddh.x_test, batch_size=128, verbose=1)
time_after_pred = datetime.now()

In [ ]:
training_time = time_after_training - time_before_training
training_time = training_time.total_seconds()

In [ ]:
pred_time = time_after_pred - time_before_pred
pred_time_avg = (pred_time.total_seconds()*1000)/len(ddh.x_test)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    pure_y_valid = backend.argmax(ddh.y_test)
    pure_y_pred = backend.argmax(predictions_valid)

    
    pure_y_valid = sess.run(pure_y_valid)
    pure_y_pred = sess.run(pure_y_pred)

In [ ]:
y_comp = [1 if i==j else 0 for i,j in zip(pure_y_pred, pure_y_valid)]
accuracy = np.mean(y_comp)

In [ ]:
print 'Accuracy: ', accuracy
print 'Training Time: ', training_time
print 'Average Time to Classify a Image: ', pred_time_avg

In [ ]:
cm = confusion_matrix(pure_y_valid, pure_y_pred)

In [ ]:
classes = ['safe driving', 'texting - right', 'talking on the phone - right', 'exting - left', 'talking on the phone - left',
          'operating the radio', 'drinking', 'reaching behind', 'hair and makeup', 'talking to passenger']
keys = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [ ]:
plot_confusion_matrix(cm, keys, normalize=False)

In [ ]:
#for i in range(100):
#    img_brute = ddh.x_manual_tests[i]
#    img = img_brute.transpose(1, 2, 0)
#    img = cv2.resize(img, (img_cols, img_rows))
#    img = img.reshape(1, 1, img_rows, img_cols)
#    predicted = model.predict_classes(img)
        
#    plt.figure()
#    plt.title(predicted)
#    plt.imshow(img_brute.reshape(120, 160), cmap='gray')